<a href="https://colab.research.google.com/github/Captmoonshot/DS-Unit-4-Spring-4-Deep-Learning/blob/master/LS_DS_Unit_4_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Unit 4 Sprint Challenge 4

## RNNs, CNNs, AutoML, and more...

In this sprint challenge, you'll explore some of the cutting edge of Data Science.

*Caution* - these approaches can be pretty heavy computationally. All problems were designed so that you should be able to achieve results within at most 5-10 minutes of runtime on Colab or a comparable environment. If something is running longer, doublecheck your approach!

## Part 1 - RNNs

Use an RNN to fit a simple classification model on tweets to distinguish from tweets from Austen Allred and tweets from Weird Al Yankovic.

Following is code to scrape the needed data (no API auth needed, uses [twitterscraper](https://github.com/taspinar/twitterscraper)):

In [1]:
!pip install twitterscraper

  Stored in directory: /root/.cache/pip/wheels/45/50/9b/70128bca07e2bf8b5ed3f504002e9e74a6eaa5e756341b6931
Successfully built twitterscraper


In [193]:
from twitterscraper import query_tweets

austen_tweets = query_tweets('from:austen', 1000)
len(austen_tweets)

INFO: queries: ['from:austen since:2006-03-21 until:2006-11-14', 'from:austen since:2006-11-14 until:2007-07-11', 'from:austen since:2007-07-11 until:2008-03-05', 'from:austen since:2008-03-05 until:2008-10-30', 'from:austen since:2008-10-30 until:2009-06-25', 'from:austen since:2009-06-25 until:2010-02-19', 'from:austen since:2010-02-19 until:2010-10-15', 'from:austen since:2010-10-15 until:2011-06-11', 'from:austen since:2011-06-11 until:2012-02-04', 'from:austen since:2012-02-04 until:2012-09-30', 'from:austen since:2012-09-30 until:2013-05-26', 'from:austen since:2013-05-26 until:2014-01-20', 'from:austen since:2014-01-20 until:2014-09-15', 'from:austen since:2014-09-15 until:2015-05-12', 'from:austen since:2015-05-12 until:2016-01-05', 'from:austen since:2016-01-05 until:2016-08-31', 'from:austen since:2016-08-31 until:2017-04-26', 'from:austen since:2017-04-26 until:2017-12-21', 'from:austen since:2017-12-21 until:2018-08-16', 'from:austen since:2018-08-16 until:2019-04-12']
INFO

181

In [194]:
austen_tweets[0].text

'I love love love working with great people.pic.twitter.com/fCKOm6Vl'

In [195]:

al_tweets = query_tweets('from:AlYankovic', 1000)
len(al_tweets)

INFO: queries: ['from:AlYankovic since:2006-03-21 until:2006-11-14', 'from:AlYankovic since:2006-11-14 until:2007-07-11', 'from:AlYankovic since:2007-07-11 until:2008-03-05', 'from:AlYankovic since:2008-03-05 until:2008-10-30', 'from:AlYankovic since:2008-10-30 until:2009-06-25', 'from:AlYankovic since:2009-06-25 until:2010-02-19', 'from:AlYankovic since:2010-02-19 until:2010-10-15', 'from:AlYankovic since:2010-10-15 until:2011-06-11', 'from:AlYankovic since:2011-06-11 until:2012-02-04', 'from:AlYankovic since:2012-02-04 until:2012-09-30', 'from:AlYankovic since:2012-09-30 until:2013-05-26', 'from:AlYankovic since:2013-05-26 until:2014-01-20', 'from:AlYankovic since:2014-01-20 until:2014-09-15', 'from:AlYankovic since:2014-09-15 until:2015-05-12', 'from:AlYankovic since:2015-05-12 until:2016-01-05', 'from:AlYankovic since:2016-01-05 until:2016-08-31', 'from:AlYankovic since:2016-08-31 until:2017-04-26', 'from:AlYankovic since:2017-04-26 until:2017-12-21', 'from:AlYankovic since:2017-12

960

In [196]:
al_tweets[0].text

"Hey @suzanneyankovic, where'd I leave my shoes?"

In [197]:
len(austen_tweets + al_tweets)

1141

Your tasks:

- Encode the characters to a sequence of integers for the model
- Get the data into the appropriate shape/format, including labels and a train/test split
- Use Keras to fit a predictive model, classifying tweets as being from Austen versus Weird Al
- Report your overall score and accuracy

For reference, the [Keras IMDB sentiment classification example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py) will be useful, as well the RNN code we used in class.

*Note* - focus on getting a running model, not on maxing accuracy with extreme data size or epoch numbers. Only revisit and push accuracy if you get everything else done!

In [0]:
# TODO - your code
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000

# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size=32


In [199]:
austen = [i.text for i in austen_tweets[:180]]
  
austen

['I love love love working with great people.pic.twitter.com/fCKOm6Vl',
 'Yes, the impact was mentioned in the S-1',
 "Risks in Uber's S1:\n\nCompetitors\nDeleteUber\nReputation\n\n2/3 are self-inflicted (and if it weren't for DeleteUber Lyft might be close to dead, so almost 3/3)",
 "In time and resources spent it's closer to two years",
 'That’s part of what this team will determine :)',
 'Our CAC rounds to zero',
 'http://ls.dev/mentor\xa0',
 "You know us now as a code school with no upfront cost and incentives aligned with students.\n\nWhat we're really building is the best place in the world to launch a new career.\n\nLambda School today is 1/1000th the size (and success) of Lambda School in the future.",
 'We\'re hiring a research and curriculum development team!\n\nThis team will help build an engine for identifying and building incredible courses at breathtaking speed.\n\nHelp us build "Lambda School for x" across dozens of verticals.https://twitter.com/armadsen/status/11164171

In [200]:
import pandas as pd
import numpy as np

austen_tweets = pd.DataFrame(austen)
austen_tweets.head()

,0
0,I love love love working with great people.pic.twitter.com/fCKOm6Vl
1,"Yes, the impact was mentioned in the S-1"
2,"Risks in Uber's S1:\n\nCompetitors\nDeleteUber\nReputation\n\n2/3 are self-inflicted (and if it weren't for DeleteUber Lyft might be close to dead, so almost 3/3)"
3,In time and resources spent it's closer to two years
4,That’s part of what this team will determine :)


In [201]:
austen_tweets['target'] = 1
austen_tweets = austen_tweets.rename({0: 'tweets'}, axis=1)
austen_tweets.head()

,tweets,target
0,I love love love working with great people.pic.twitter.com/fCKOm6Vl,1
1,"Yes, the impact was mentioned in the S-1",1
2,"Risks in Uber's S1:\n\nCompetitors\nDeleteUber\nReputation\n\n2/3 are self-inflicted (and if it weren't for DeleteUber Lyft might be close to dead, so almost 3/3)",1
3,In time and resources spent it's closer to two years,1
4,That’s part of what this team will determine :),1


In [202]:
al = [i.text for i in al_tweets[:959]]
  
al

["Hey @suzanneyankovic, where'd I leave my shoes?",
 'R.I.P. Ed McMahon  http://tinyurl.com/l5ekm8',
 'http://twitpic.com/83r19 - West Hollywood yogurt shop, take 2',
 'Breakfast in bed - Happy Fathers Day!  http://twitpic.com/80fvh',
 'http://twitpic.com/7x4yt - Roadside billboard sighting #2',
 'http://twitpic.com/7u8mk - How can I get Seth Green to stop riding my Segway around the house??',
 'RT @ScottAukerman: CDR Radio today at 12N PST with guest @alyankovic! Plus @paulscheer & @dougbenson! http://indie1031.com to listen LIVE',
 "Please welcome my friend Drew Carey to Twitter - he's @TPIRhost.",
 "Ever have your toenails painted by a 6-year-old?  It's awesome!  http://twitpic.com/7qosk",
 "http://twitpic.com/7oxwk - I'm pretty sure this is my first time ever on a billboard...",
 'RT @michaelianblack: "If you\'d like to read my thoughts on Weird Al: http://tinyurl.com/mcjt8g. WARNING: NO NUDITY!!! SOME WEIRD AL!!!"',
 'Ustream broadcast commencing... NOW!!!  Tag questions with #Wei

In [203]:
al_tweets = pd.DataFrame(al)
al_tweets.head()

,0
0,"Hey @suzanneyankovic, where'd I leave my shoes?"
1,R.I.P. Ed McMahon http://tinyurl.com/l5ekm8
2,"http://twitpic.com/83r19 - West Hollywood yogurt shop, take 2"
3,Breakfast in bed - Happy Fathers Day! http://twitpic.com/80fvh
4,http://twitpic.com/7x4yt - Roadside billboard sighting #2


In [204]:
al_tweets['target'] = 0
al_tweets = al_tweets.rename({0: 'tweets'}, axis=1)
al_tweets.head()

,tweets,target
0,"Hey @suzanneyankovic, where'd I leave my shoes?",0
1,R.I.P. Ed McMahon http://tinyurl.com/l5ekm8,0
2,"http://twitpic.com/83r19 - West Hollywood yogurt shop, take 2",0
3,Breakfast in bed - Happy Fathers Day! http://twitpic.com/80fvh,0
4,http://twitpic.com/7x4yt - Roadside billboard sighting #2,0


In [205]:
df = pd.concat([austen_tweets, al_tweets])
df.head()

,tweets,target
0,I love love love working with great people.pic.twitter.com/fCKOm6Vl,1
1,"Yes, the impact was mentioned in the S-1",1
2,"Risks in Uber's S1:\n\nCompetitors\nDeleteUber\nReputation\n\n2/3 are self-inflicted (and if it weren't for DeleteUber Lyft might be close to dead, so almost 3/3)",1
3,In time and resources spent it's closer to two years,1
4,That’s part of what this team will determine :),1


In [206]:
df.shape

(1139, 2)

In [220]:
from sklearn.utils import shuffle
df= shuffle(df)
df.head()

,tweets,target
132,"In honor of the Olympics, I’m continuing my “kooky Canadians” interview series: http://bit.ly/bdr0PI",0
296,Cool! Just found out that every new iPad purchase comes with a free deep tissue massage!pic.twitter.com/WshmBnZlTF,0
554,"As a matter of fact, you are - but don’t tell anybody!",0
72,This will end in tearspic.twitter.com/nyaiZmZqpg,1
753,"Yes, @stephenathome, it HAS been a rather tough week. http://instagram.com/p/rFkgDcGuzV/",0


In [208]:
from sklearn.model_selection import train_test_split

y = df['target']
X = df.drop('target', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')


854 train sequences
285 test sequences


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
X_train_vectorized = count.fit_transform(X_train)

In [0]:
X_test_vectorized = count.transform(X_test)

In [187]:
X_train = sequence.pad_sequences(X_train_vectorized)

TypeError: ignored

In [188]:
X_test = sequence.pad_sequences(X_test_vectorized)

TypeError: ignored

In [189]:
print('x_train shape:', X_train.shape)
print('x_test shape:', X_test.shape)

x_train shape: (854, 1)
x_test shape: (285, 1)


In [85]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


Train...
Train on 854 samples, validate on 285 samples
Epoch 1/15


ValueError: ignored

Conclusion - RNN runs, and gives pretty decent improvement over a naive "It's Al!" model. To *really* improve the model, more playing with parameters, and just getting more data (particularly Austen tweets), would help. Also - RNN may well not be the best approach here, but it is at least a valid one.

## Part 2- CNNs

Time to play "find the frog!" Use Keras and ResNet50 to detect which of the following images contain frogs:

In [177]:
!pip install google_images_download

    100% |████████████████████████████████| 911kB 12.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/d2/23/84/3cec6d566b88bef64ad727a7e805f6544b8af4a8f121f9691c
Successfully built google-images-download


In [178]:
from google_images_download import google_images_download

response = google_images_download.googleimagesdownload()
arguments = {"keywords": "animal pond", "limit": 5, "print_urls": True}
absolute_image_paths = response.download(arguments)


Item no.: 1 --> Item name = animal pond
Evaluating...
Starting Download...
Image URL: https://www.enchantedlearning.com/pgifs/Pondanimals.GIF
Completed Image ====> 1. pondanimals.gif
Image URL: https://i.ytimg.com/vi/NCbu0TND9vE/hqdefault.jpg
Completed Image ====> 2. hqdefault.jpg
Image URL: https://pklifescience.com/staticfiles/articles/images/PKLS4116_inline.png
Completed Image ====> 3. pkls4116_inline.png
Image URL: https://pixnio.com/free-images/fauna-animals/reptiles-and-amphibians/alligators-and-crocodiles-pictures/alligator-animal-on-pond.jpg
Completed Image ====> 4. alligator-animal-on-pond.jpg
Image URL: https://www.nwf.org/-/media/NEW-WEBSITE/Programs/Garden-for-Wildlife/amphibian_bronze-frog_Julia-Bartosh_400x267.ashx
Completed Image ====> 5. amphibian_bronze-frog_julia-bartosh_400x267.ash

Errors: 0



At time of writing at least a few do, but since the Internet changes - it is possible your 5 won't. You can easily verify yourself, and (once you have working code) increase the number of images you pull to be more sure of getting a frog. Your goal is to validly run ResNet50 on the input images - don't worry about tuning or improving the model.

*Hint* - ResNet 50 doesn't just return "frog". The three labels it has for frogs are: `bullfrog, tree frog, tailed frog`

*Stretch goal* - also check for fish.

In [179]:
# TODO - your code!
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps  # https://pillow.readthedocs.io/en/stable/
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

image_path_list = absolute_image_paths['animal pond']

def resize_image(filename, new_width=256, new_height=256):
  pil_image = Image.open(filename)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert('RGB')
  pil_image_rgb.save(filename, format='JPEG', quality=90)

for path in image_path_list:
  resize_image(path, 224, 224)
  
def process_img_path(img_path):
  return image.load_img(img_path, target_size=(224, 224))

def interpret(img):
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  model = ResNet50(weights='imagenet')
  features = model.predict(x)
  results = decode_predictions(features, top=3)[0]
  return results

all_predictions = []
for i, image_path in enumerate(image_path_list):
  results = interpret(process_img_path(image_path))
  all_predictions.append(results)
  print(results)

40960/35363 [==================================] - 0s 1us/step
[('n06359193', 'web_site', 0.8918389), ('n04404412', 'television', 0.08547714), ('n04152593', 'screen', 0.006043785)]
[('n01443537', 'goldfish', 0.57714343), ('n02536864', 'coho', 0.3193861), ('n01630670', 'common_newt', 0.020654997)]
[('n04243546', 'slot', 0.9574489), ('n04476259', 'tray', 0.014238177), ('n03908618', 'pencil_box', 0.007384028)]
[('n01698640', 'American_alligator', 0.59087384), ('n01737021', 'water_snake', 0.13598265), ('n01697457', 'African_crocodile', 0.07479092)]
[('n01641577', 'bullfrog', 0.66036487), ('n01644900', 'tailed_frog', 0.3108625), ('n01630670', 'common_newt', 0.012726643)]


## Part 3 - AutoML

Use [TPOT](https://github.com/EpistasisLab/tpot) to fit a predictive model for the King County housing data, with `price` as the target output variable.

In [87]:
!pip install tpot

    100% |████████████████████████████████| 81kB 3.9MB/s 
    100% |████████████████████████████████| 942kB 17.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/22/ea/bf/dc7c8a2262025a0ab5da9ef02282c198be88902791ca0c6658
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built deap stopit


In [88]:
!wget https://raw.githubusercontent.com/ryanleeallred/datasets/master/kc_house_data.csv

--2019-04-12 16:08:15--  https://raw.githubusercontent.com/ryanleeallred/datasets/master/kc_house_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2515206 (2.4M) [text/plain]
Saving to: ‘kc_house_data.csv’

kc_house_data.csv   100%[===================>]   2.40M  --.-KB/s    in 0.07s   

2019-04-12 16:08:15 (32.7 MB/s) - ‘kc_house_data.csv’ saved [2515206/2515206]



In [89]:
!head kc_house_data.csv

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
"7129300520","20141013T000000",221900,3,1,1180,5650,"1",0,0,3,7,1180,0,1955,0,"98178",47.5112,-122.257,1340,5650
"6414100192","20141209T000000",538000,3,2.25,2570,7242,"2",0,0,3,7,2170,400,1951,1991,"98125",47.721,-122.319,1690,7639
"5631500400","20150225T000000",180000,2,1,770,10000,"1",0,0,3,6,770,0,1933,0,"98028",47.7379,-122.233,2720,8062
"2487200875","20141209T000000",604000,4,3,1960,5000,"1",0,0,5,7,1050,910,1965,0,"98136",47.5208,-122.393,1360,5000
"1954400510","20150218T000000",510000,3,2,1680,8080,"1",0,0,3,8,1680,0,1987,0,"98074",47.6168,-122.045,1800,7503
"7237550310","20140512T000000",1.225e+006,4,4.5,5420,101930,"1",0,0,3,11,3890,1530,2001,0,"98053",47.6561,-122.005,4760,101930
"1321400060","20140627T000000",257500,3,2.25,1715,6819,"2",0,0,3,7,1715,0,1995,0,"98003",47.3097,-122.327,2238,6819
"2

As with previous questions, your goal is to run TPOT and successfully run and report error at the end.  Also, in the interest of time, feel free to choose small `generation=1` and `population_size=10` parameters so your pipeline runs efficiently and you are able to iterate and test.

*Hint* - you'll have to drop and/or type coerce at least a few variables to get things working. It's fine to err on the side of dropping to get things running, as long as you still get a valid model with reasonable predictive power.

In [105]:
# TODO - your code!
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor

housing = pd.read_csv('kc_house_data.csv', parse_dates=['date'], na_values=['-1.#IND', '1.#QNAN', '1.#IND', '-1.#QNAN', '#N/A','N/A', 'NA', '#NA', 'NULL', 'NaN', '-NaN', 'nan', '-nan', '?'])
housing.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [106]:
housing.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [96]:
housing.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [0]:
housing = housing.dropna()

In [108]:
housing.dtypes

id               int64         
date             datetime64[ns]
price            float64       
bedrooms         int64         
bathrooms        float64       
sqft_living      int64         
sqft_lot         int64         
floors           float64       
waterfront       int64         
view             int64         
condition        int64         
grade            int64         
sqft_above       int64         
sqft_basement    int64         
yr_built         int64         
yr_renovated     int64         
zipcode          int64         
lat              float64       
long             float64       
sqft_living15    int64         
sqft_lot15       int64         
dtype: object

In [0]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

In [111]:
y = housing['price'].values
X = housing.drop(['price', 'id','date'], axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

tpot = TPOTRegressor(generations=1, population_size=10, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Generation 1 - Current best internal CV score: -18669668811.297523

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=4, min_samples_split=2, n_estimators=100)
-16837761793.66108


## Part 4 - More...

Answer the following questions, with a target audience of a fellow Data Scientist:

- What do you consider your strongest area, as a Data Scientist?
- What area of Data Science would you most like to learn more about, and why?
- Where do you think Data Science will be in 5 years?

A few sentences per answer is fine - only elaborate if time allows.

My strongest area as a Data Scientis is statistical curiosity.

The area I'm most interested in is how Data Science can actually help people do things better, as in more effectively and not just efficiently.

I have no idea what Data Science will be like in 5 years, but from what I've observed there may be an oversupply of job-seekers in the area.  

The internet, web 2.0, smartphones ended up changing our entire society.  But ML hasn't had that effect, which is why I think there's so much interest in Applied ML.

I also think there's a lot of hype in the space.  Things like self-driving is definitely cool, but it's too niche.  I think we're in the 4th innning of a 9 inning game, but my hope is that Data Science/ML doesn't end up overpromising and under-delivering.  

Thank you for your hard work, and congratulations! You've learned a lot, and should proudly call yourself a Data Scientist.